In [64]:
import sys
sys.path.append('../main')
import sql
import pandas as pd
from functools import reduce
from tabulate import tabulate

In [65]:
q='''select a.id, datediff(s.date,first_detected) as lifespan, me.memory, p.name from alert a
        join project p on a.project_id = p.id
        join snapshot s on a.last_snapshot_id=s.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        join actionability ac on ac.alert_id=a.id
        where is_invalid=0
        and status='Fixed' 
        and actionability=1'''
df=pd.DataFrame(sql.execute(q))
ta=df.groupby('name')[['lifespan']].median()
ta=ta.rename(columns={'lifespan':'actionable'})
ta

,actionable
name,
!CHAOS Control System,136.5
Chromium EC,93.0
Firefox,124.0
Kodi,62.0
LibreOffice,18.0
Linux,258.0
OpenCV,345.5
Samba,70.0
Thunderbird,915.5


In [66]:
q='''select a.id, datediff(s.date,first_detected) as lifespan, me.memory, p.name from alert a
        join project p on a.project_id = p.id
        join snapshot s on a.last_snapshot_id=s.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        join actionability ac on ac.alert_id=a.id
        where is_invalid=0
        and status='Fixed' 
        and actionability=1
        and me.memory =1 '''
df=pd.DataFrame(sql.execute(q))
tma=df.groupby('name')[['lifespan']].median()
tma=tma.rename(columns={'lifespan':'memory'})
tma

,memory
name,
!CHAOS Control System,48
Chromium EC,247
Firefox,44
Kodi,6
LibreOffice,16
Linux,257
OpenCV,342
Samba,143
Thunderbird,1591


In [67]:
q='''select a.id, datediff(s.date,first_detected) as lifespan, me.memory, p.name from alert a
        join project p on a.project_id = p.id
        join snapshot s on a.last_snapshot_id=s.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        join actionability ac on ac.alert_id=a.id
        where is_invalid=0
        and status='Fixed' 
        and actionability=1
        and me.memory is null '''
df=pd.DataFrame(sql.execute(q))
tnma=df.groupby('name')[['lifespan']].median()
tnma=tnma.rename(columns={'lifespan':'non_memory'})
tnma

,non_memory
name,
!CHAOS Control System,203
Chromium EC,70
Firefox,154
Kodi,82
LibreOffice,18
Linux,258
OpenCV,380
Samba,37
Thunderbird,490


In [68]:
df=reduce(lambda x,y : pd.merge(x,y,on='name'),[ta,tma, tnma])

In [69]:
# df=df.reset_index()
# df=df.sort_values('actionable',ascending=False)
# df=df.rename(columns={'name':'Project', 'actionable':'Lifespan of all actionable alerts (days)',
# 'memory': 'Lifespan of Memory alerts (days)','non_memory': 'Lifespan of Non-memory alerts (days)'})
# print(tabulate(df, tablefmt='pipe',showindex=False, headers='keys'))

In [70]:
from scipy.stats import mannwhitneyu
stat, p= mannwhitneyu(df['memory'], df['non_memory'])
stat, p, df['memory'].median(), df['non_memory'].median()

(53.0, 0.32288191015248613, 77.0, 119.0)

In [71]:
q='''select a.id, datediff(s.date,first_detected) as lifespan, me.memory, p.name from alert a
        join project p on a.project_id = p.id
        join snapshot s on a.last_snapshot_id=s.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        join actionability ac on ac.alert_id=a.id
        where is_invalid=0
        and status='Fixed' 
        and classification='Bug' '''
df=pd.DataFrame(sql.execute(q))
ta=df.groupby('name')[['lifespan']].median()
ta=ta.rename(columns={'lifespan':'marked_bug'})
q='''select a.id, datediff(s.date,first_detected) as lifespan, me.memory, p.name from alert a
        join project p on a.project_id = p.id
        join snapshot s on a.last_snapshot_id=s.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        join actionability ac on ac.alert_id=a.id
        where is_invalid=0
        and status='Fixed' 
        and classification='Bug' 
        and me.memory =1 '''
df=pd.DataFrame(sql.execute(q))
tma=df.groupby('name')[['lifespan']].median()
tma=tma.rename(columns={'lifespan':'memory'})
q='''select a.id, datediff(s.date,first_detected) as lifespan, me.memory, p.name from alert a
        join project p on a.project_id = p.id
        join snapshot s on a.last_snapshot_id=s.id
        join memory_error me on a.alert_type_id = me.alert_type_id
        join actionability ac on ac.alert_id=a.id
        where is_invalid=0
        and status='Fixed' 
        and classification='Bug' 
        and me.memory is null '''
df=pd.DataFrame(sql.execute(q))
tnma=df.groupby('name')[['lifespan']].median()
tnma=tnma.rename(columns={'lifespan':'non_memory'})
df=reduce(lambda x,y : pd.merge(x,y,on='name'),[ta,tma, tnma])
df

,marked_bug,memory,non_memory
name,,,
Chromium EC,377,429.0,367.0
Firefox,64,63.0,71.5
Kodi,2,2.0,2.0
LibreOffice,15,21.0,6.0
Linux,190,134.0,252.0
OpenCV,5,850.5,5.0
Samba,665,665.0,911.0
Thunderbird,1385,1779.0,970.0
VTK,24,24.0,30.0


In [73]:
df=df.reset_index()
df=df.sort_values('marked_bug',ascending=False)
df=df.rename(columns={'name':'Project', 'marked_bug':'Lifespan of alerts marked as bug (days)',
'memory': 'Lifespan of Memory alerts (days)','non_memory': 'Lifespan of Non-memory alerts (days)'})
print(tabulate(df, tablefmt='pipe',showindex=False, headers='keys'))

| Project      |   Lifespan of alerts marked as bug (days) |   Lifespan of Memory alerts (days) |   Lifespan of Non-memory alerts (days) |
|:-------------|------------------------------------------:|-----------------------------------:|---------------------------------------:|
| Thunderbird  |                                      1385 |                             1779   |                                  970   |
| Samba        |                                       665 |                              665   |                                  911   |
| Chromium EC  |                                       377 |                              429   |                                  367   |
| Linux        |                                       190 |                              134   |                                  252   |
| Firefox      |                                        64 |                               63   |                                   71.5 |
| ovirt-engine |           

In [72]:
from scipy.stats import mannwhitneyu
stat, p= mannwhitneyu(df['memory'], df['non_memory'])
stat, p, df['memory'].median(), df['non_memory'].median()

(44.5, 0.35267568603175237, 98.5, 60.5)